## Building dependency based co-occurrence matrix 

This is a compiled examples of how to build co-occurrence matrix using dependency relations.

### Note
- Fixed the lowercase issue for CONLLU 
- If CONLLU is the format used for the corpus and the POS is missing, then instead xpos (if available) will be replaced as POS. 
- Analogies currently does not support Token type. Only works if one field (in this case "form" or "lemma") is used for target_vocabulary. 
- Even if parameter deprel for dependency_context does not list dependency relation denoting prepositions such as "prep" or "case" will be included. 
    


In [ ]:
from IPython.display import display, HTML
import mangoes
import nltk
import string 
import pprint 
import os 
import datetime 
import logging 
from collections import Counter 

Read more for 

https://github.com/UniversalDependencies/UD_English-EWT


In [ ]:
date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")

In [ ]:
# Or specify date to saved ones
# eg: date = "2021-03-24-15-23"

In [ ]:
OUTPUT_PATH = os.path.join(os.path.abspath(''), "output/{}".format(date))
if not os.path.exists(OUTPUT_PATH):
    print("made a dir: ", OUTPUT_PATH)
    os.makedirs(OUTPUT_PATH)   

In [ ]:
date

In [ ]:
# for logging 
logging.basicConfig(level=logging.DEBUG, 
                    filename=f"{OUTPUT_PATH}/log", 
                    filemode="a+",
                    format="%(asctime)s;%(levelname)s;%(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S"
                   )

# Build Embedding 
### Corpus

In [ ]:
# Specify the path to the data contained directory/file 
UD_English_EWT = "../../UD_English-EWT-master/corpus"
# UD_English_EWT =  "../../UD_English-EWT-master/corpus/en_ewt-ud-train.conllu"
# WIKI = "../../wikipedia_data/treebank.conllu"

In [ ]:
logging.info(f"Corpus path: {UD_English_EWT}")

NOTE: When loading dataset to `Corpus`, set `ignore_punctuation` to `False` if `DependencyContext` is used to build co-occurrence matrix

In [ ]:
corpus = mangoes.Corpus(UD_English_EWT, 
                        reader=mangoes.corpus.CONLLU, 
                        language="English",
                        lower=True, 
                        ignore_punctuation=False) 

In [ ]:
logging.info("Corpus has {} sentences, {} words, {} unique tokens".format(corpus.nb_sentences, corpus.size, len(corpus.words_count)))

In [ ]:
corpus.describe()

In [ ]:
#Save corpus 
corpus_metadata = os.path.join(OUTPUT_PATH, ".corpus")
print("Saved Corpus")
corpus.save_metadata(corpus_metadata)

In [ ]:
# or load 
# corpus = mangoes.Corpus.load_from_metadata(corpus_metadata)

### Vocabulary
#### -- Target Words --

In [ ]:
logging.info("Building target words...")

In [ ]:
stopwords_filter_lemma = mangoes.corpus.remove_elements(nltk.corpus.stopwords.words('english'), attribute="lemma")

In [ ]:
stopwords_filter = mangoes.corpus.remove_elements(nltk.corpus.stopwords.words('english'))

#### Option 1: Use Lemma and POS as targetwords

In [ ]:
target_vocabulary = corpus.create_vocabulary(attributes=("lemma","POS"), 
                                              filters = [ stopwords_filter_lemma,  # removes stopwords 
                                                         mangoes.corpus.remove_most_frequent(100), # removes words that are frequent more than 100
                                                         mangoes.corpus.remove_least_frequent(2)]) # removes words that are frequent less than 2

#### Option 2: Only keep lemma as targetwords

In [ ]:
target_vocabulary = corpus.create_vocabulary(attributes="lemma", 
                                              filters = [ stopwords_filter, 
                                                         mangoes.corpus.remove_most_frequent(100),
                                                         mangoes.corpus.remove_least_frequent(2)])

#### Option 3: Use Lemma and POS as targetwords + keep only POS of "NOUN", "VERB", "ADJ"

In [ ]:
pos_filter_target = mangoes.corpus.filter_by_attribute("POS", ["NOUN", "VERB", "ADJ"])

In [ ]:
# must include POS as one of attributes if POS filter is used
target_vocabulary = corpus.create_vocabulary(attributes=("lemma","POS"), 
                                            filters = [ stopwords_filter_lemma, 
                                                        pos_filter_target,
                                                        mangoes.corpus.remove_most_frequent(100),
                                                        mangoes.corpus.remove_least_frequent(2)],)

#### Check target vocaulary

In [ ]:
print(f"{len(target_vocabulary)} words will be used as target vocabulary")

In [ ]:
logging.info(f"Target Words: {len(target_vocabulary)}")

In [ ]:
# check all the target vocabulary
target_vocabulary._index_word

In [ ]:
# Unique counts for POS if POS is in target vocabulary 

if "POS" in target_vocabulary.entity:
    POS_target_count = Counter(getattr(token, "POS") for token in target_vocabulary._index_word)
    print(POS_target_count)

#### -- Context Words --

In [ ]:
logging.info("Building Context words...")

#### Option 1: Only keep lemma 

In [ ]:
context_vocabulary = corpus.create_vocabulary(attributes=("lemma"), 
                                              filters = [ stopwords_filter])

#### Option 2: Keep lemma and POS with "VERB" or "ADJ"

In [ ]:
pos_filter_context = mangoes.corpus.filter_by_attribute("POS",["NOUN", "VERB", "ADJ"])

In [ ]:
context_vocabulary = corpus.create_vocabulary(attributes=("lemma","POS"), 
                                            filters = [ stopwords_filter_lemma,
                                                        pos_filter_context])

#### Check context vocabulary

In [ ]:
print(f"{len(context_vocabulary)} words will be used as target vocabulary")

In [ ]:
logging.info(f"Context words: {len(context_vocabulary)}")

In [ ]:
context_vocabulary._index_word

In [ ]:
if "POS" in context_vocabulary.entity:
    POS_context_count = Counter(getattr(token, "POS") for token in context_vocabulary._index_word)
    print(POS_context_count)

### Context 

#### Option 1: consider dependency relation  as undirected by setting directed parameter 

In [ ]:
# NOTE: entity needs to match with context_vocabulary 
dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma", "POS"), 
                                                            labels=True,
                                                            collapse=True, 
                                                            vocabulary=context_vocabulary,
                                                            directed=False)

#### Option 2: consider only dependency relation of "advmod" or "nsubj"
Note if collapse is set to True, then case will be included as well. 


This will result in changed number of context words when co-occurrence matrix is built 

In [ ]:
dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma"), 
                                                            labels=False,
                                                            collapse=True, 
                                                            vocabulary=context_vocabulary,
                                                            deprel_keep = ("advmod", "nsubj"),
                                                            directed=False)

#### Option 3 adding depth to consider indirected dependency relation

In [ ]:
dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma","POS"), 
                                                            labels=True,
                                                            collapse=True, 
                                                            vocabulary=context_vocabulary,
                                                            directed=False,
                                                            depth=2
                                                            )

#### Option 4: Combining option 2 and 3

If `depth` > 1 and `deprel_keep` is specified then `deprel_keep` will be counted as long as indirected dependency relations has one relation from `deprel_keep`.


eg. If `depth=2` and `deprel_keep=("nsubj","amod")` then  `"{'lemma': 'muslim', 'POS': 'ADJ'}/nsubj_amod",` will be included as context words since one of the dependency relation `nsubj` and `amod` is in `deprel_keep`

In [ ]:
dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma","POS"), 
                                                            labels=True,
                                                            collapse=True, 
                                                            vocabulary=context_vocabulary,
                                                            directed=False,
                                                            depth=2,
                                                            deprel_keep=("advmod", "nsubj")
                                                            )

#### Option 5: Basis mapping 

So far all the previous options passes `label` as `True` thus, co-occurrence matrix context words considers unique token + dependency relation. However, we can apply basis mapping, meaning decouple the observed syntactic context from the final co-occurrence matrix. By setting the `label` to `False` while defining the context by using the DependnencyBasedContext, we still have all those constrains specified by other arguments but we will only count the unique tokens. 

e.g.: if `label=False` and `depth=2` then `{'lemma':'apple', 'POS':'NOUN'}/obj` and `{'lemma':'apple', 'POS':'NOUN'}/obj_mod` will mapped to context vocabulary of `{'lemma':'apple', 'POS':'NOUN'}`

In [ ]:
dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma","POS"), 
                                                            labels=False,
                                                            collapse=False, 
                                                            vocabulary=context_vocabulary,
                                                            directed=False,
                                                            depth=3                                                          
                                                            )

#### Option 6: Weight

Instead of counting the frequency and equally adding 1, you can also diffierentiate the relative importance of different paths. 

By setting `weight=True`, paths can weigh their respective contributions differently. It assigns a value of 1 to paths of length 1 and fractions to longer paths: 1 / len(path_lengths)

Instead, if you want to use a more linguistically-informed path value function you can pass a dictionary to `weight_scheme` parameters. It will assign specific weight to grammatical relations, without considering their length:. 
e.g.: `weight_scheme={"nsubj":5}` means any paths that have `nsubj` as one of their dependency relations will get value of 5.  


If the `weight` is not set but `weight_scheme` is then it will internally assign `weight=True`. 

In [ ]:
dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma","POS"), 
                                                            labels=True,
                                                            collapse=True, 
                                                            vocabulary=context_vocabulary,
                                                            directed=False,
                                                            depth=2, 
                                                            weight=True,
                                                            weight_scheme={"nsubj":5, "amod":4, "nmod":3}
                                                            )

In [ ]:
# # save vocabularies 
target_vocabulary_file_name = "vocabulary_{}_target_words".format(len(target_vocabulary))
context_vocabulary_file_name = "vocabulary_{}_context_words".format(len(context_vocabulary))

# # or load by specify saved vocabulary files 

In [ ]:
# # save
target_vocabulary.save(OUTPUT_PATH, name=target_vocabulary_file_name)
context_vocabulary.save(OUTPUT_PATH, name=context_vocabulary_file_name)

In [ ]:
# load vocabs
# target_vocabulary = mangoes.Vocabulary.load("output", target_vocabulary_file_name)
# context_vocabulary = mangoes.Vocabulary.load("output", context_vocabulary_file_name)

### Cooccurrence Matrix 

In [ ]:
coocc_count = mangoes.counting.count_cooccurrence(corpus,  
                                                target_vocabulary, 
                                                context=dependency_context,
                                                )

coocc_count.pprint(display=display)

In [ ]:
print("target words")
coocc_count.words[:20]

In [ ]:
print("context words")
coocc_count.contexts_words[:20]

In [ ]:
coocc_count.shape

#### Get closes words

In [ ]:
i = 8

print("target word: ", target_vocabulary[i])
print("close words")

# NOTE: make sure POS you want to get exist in target_vocabulary.  
# only_verb = [x for x in coocc_count.get_closest_words(target_vocabulary[i], nb=100) if x[0].POS in ['VERB']]
# pprint.pprint(only_verb)

pprint.pprint(coocc_count.get_closest_words(target_vocabulary[i], 30)) # default nb is 10

In [ ]:
# Count unique Dependency relation in built co-occurrence matrix
# Run below only if labels=True was used
if dependency_context.labels:
    dep_rel_count = Counter(word.split("/")[-1] for word in coocc_count.contexts_words)
    pprint.pprint(dep_rel_count.most_common())

### Weighting 

In [ ]:
ppmi = mangoes.weighting.PPMI()
svd = mangoes.reduction.SVD(dimensions=300)

In [ ]:
embeddings = mangoes.create_representation(coocc_count, weighting=ppmi, reduction=svd)

In [ ]:
embeddings.pprint(display=display)

In [ ]:
# or you need to 
i = 100

print("target word: ", target_vocabulary[i])
print("close words")

only_verb = [x for x in embeddings.get_closest_words(target_vocabulary[i], nb=1000) if x[0].POS in ['VERB']]
pprint.pprint(only_verb)

# pprint.pprint(embeddings.get_closest_words(target_vocabulary[i], 30)) # default nb is 10

In [ ]:
# # Save the embeddings 
embedding_path = os.path.join(OUTPUT_PATH,
                              "embeddings/ppmi_svd_{}target_words_deprel".format(len(target_vocabulary)))

In [ ]:
embeddings.save(embedding_path)

In [ ]:
# load embedding 
# embeddings = mangoes.Embeddings.load(embedding_path)

# Explore Embedding 

In [ ]:
import pandas as pd 

### Closest Words 

#### Similarities 

In [ ]:
sims = {0: "cityblock",
        1: "cosine", 
        2: "euclidean", 
        3: "l1", 
        4:"l2", 
        5:"manhattan",
        6:"braycurtis", 
        7:"canberra", 
        8:"chebyshev", 
        9:"correlation", 
        10:"dice", 
        11:"hamming", 
        12:"jaccard", 
        13:"kulsinski", 
        14:"mahalanobis", 
       15: "minkowski", 
       16: "rogerstanimoto", 
       17: "russellrao", 
       18: "seuclidean", 
        19:"sokalmichener", 
        20:"sokalsneath", 
       21: "sqeuclidean", 
       22: "yule"}


In [ ]:
i = 2
nb = 5

In [ ]:
result = {word: pd.Series([w for w, _ in embeddings.get_closest_words(word, nb=nb, metric=sims[i])], index=range(1,nb+1))
          for word in embeddings.words[100:200:10]}
result_df = pd.DataFrame(result).transpose()

In [ ]:
print(f"similarity measure: {sims[i]}")
display(result_df)

### Analogies 

In [ ]:
# This only works if target vocabulary is not type Token. 
question = "king queen male"
answer = "female"

# If target vocabulary is a Token then uncomment following instead
# from mangoes.utils.reader import AnnotatedSentenceGenerator 
# Token = AnnotatedSentenceGenerator.Token

# question = [
#     Token(lemma="queen", POS="NOUN"),
#     Token(lemma="king", POS="NOUN"),
#     Token(lemma="male", POS="NOUN")
# ]
# answer = Token(lemma="female", POS="NOUN")

# filter_token = mangoes.vocabulary.create_token_filter(["lemma", "POS"])

# question = list(map(filter_token, question))
# answer = filter_token(answer)

In [ ]:
# Collocation 2: Oper1
question = "help give help"
answer = "offer"

In [ ]:
# Check what words are available in our embeddings #
for w in (question.split() + [answer]):
    try:
        embeddings.words.word_index[w]
        print(f"'{w}' exists")
    except KeyError:
        print(f"'{w}' doesn't exist in embedding")

In [ ]:
# You can resolve analogy according to a representation using the analogy() method
# Here, we will display the results of some examples :
for analogy in [question]:
    print(analogy, '->', embeddings.analogy(analogy,5).using_cosadd)
    print(analogy, '->', embeddings.analogy(analogy,5).using_cosmul)

### Visualization

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import mangoes.visualize
plt.figure()

# 1. distances between the words
ax = plt.subplot(221, projection='polar')
mangoes.visualize.plot_distances(embeddings, ax)

# 2. isotropy
ax = plt.subplot(222)
mangoes.visualize.plot_isotropy(embeddings, ax)

# 3. t-sne
plt.subplot(212)
mangoes.visualize.plot_tsne(embeddings)

plt.show()


# Evalutaion 

In [ ]:
# # Evaluate
import mangoes.evaluation.analogy

google_dataset = mangoes.evaluation.analogy.GOOGLE
msr_dataset = mangoes.evaluation.analogy.MSR

analogy_evaluation = mangoes.evaluation.analogy.Evaluation(embeddings, google_dataset, msr_dataset)
print(analogy_evaluation.get_report())